In [1]:
from pathlib import Path
from pyraphtory.context import PyRaphtory

In [2]:
pr = PyRaphtory(spout_input=Path('/tmp/lotr.csv'), builder_script=Path('builder.py'),  builder_class='LotrGraphBuilder', mode='batch', logging=True).open()

b'WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.\n'
b'12:45:36.692 [io-compute-32] DEBUG com.raphtory.utils.FileUtils$ - Found single file /tmp/lotr.csv matching criteria.\n'
b"12:45:36.693 [io-compute-32] DEBUG com.raphtory.utils.FileUtils$ - Temporary directory '/tmp/raphtory-file-spout/-7183574349260151695' successfully created.\n"
b"12:45:36.693 [io-compute-32] DEBUG com.raphtory.spouts.FileSpout - Spout: Found a new file '/lotr.csv' to process.\n"
b"12:45:36.694 [io-compute-32] DEBUG com.raphtory.spouts.FileSpout - Spout: Attempting to hard link file '/tmp/lotr.csv' -> '/tmp/raphtory-file-spout/-7183574349260151695/lotr.csv'.\n"
b"12:45:36.694 [io-compute-32] INFO  com.raphtory.spouts.FileSpout - Spout: Processing file 'lotr.csv' ...\n"
b'12:45:36.768 [io-compute-32] INFO  com.raphtory.internals.management.Prometheus$ - Prometheus started on port /0:0:0:0:0:0:0:0:9999\n'
b'12:45:37.233 [spawner-akka.actor.default-dispatcher-3] INFO  

In [3]:
from pyraphtory.algo import Vertex, Iterate, Step

In [4]:
PR_LABEL = 'prlabel'

class PGStep1(Step):
    def eval(self, v: Vertex):
        initLabel = 1.0
        v[PR_LABEL] = initLabel
        out_degree = v.out_degree()
        if out_degree > 0:
            msg = initLabel / out_degree
            v.message_outgoing_neighbours(msg)

class PGIterate1(Iterate):
    def __init__(self, iterations: int, execute_messaged_only: bool, damping_factor: float = 0.85):
        super().__init__(iterations, execute_messaged_only)
        self.damping_factor = damping_factor

    def eval(self, v: Vertex):
        current_label = v[PR_LABEL]
        queue = v.message_queue()
        summed_queue = sum(queue)
        new_label = (1 - self.damping_factor) + self.damping_factor * summed_queue
        v[PR_LABEL] = new_label

        out_degree = v.out_degree()

        if out_degree > 0:
            v.message_outgoing_neighbours(new_label / out_degree)

        if abs(new_label - current_label) < 0.00001:
            v.vote_to_halt()


In [5]:
rg = pr.graph()

In [15]:
tracker = rg.at(32674) \
                .past() \
                .step(PGStep1()) \
                .iterate(PGIterate1(iterations=100, execute_messaged_only=False)) \
                .select([PR_LABEL]) \
                .write_to_file("/tmp/jupyter_pagerank")

In [7]:
print(tracker)

In [ ]:
from threading import Thread

def loopy(rg):
    for _ in rg.log_lines(False):
        pass
    
t = Thread(target=loopy, args=(pr, ))

In [ ]:
t.start()

In [ ]:
tracker.wait_for_job()

In [13]:
t.start()

In [14]:
tracker.wait_for_job()